<a href="https://colab.research.google.com/github/ryan9453/linebottest/blob/main/%E8%A9%A6%E5%81%9A%E6%A9%9F%E5%99%A8%E4%BA%BA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [92]:
'''

資料 mapping 至google drive

把資料寫在/content/drive

即可保存在 google drive內

'''
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [93]:
# 安裝套件
!pip install line-bot-sdk flask flask-ngrok

In [94]:
'''
引用套件
'''

# 引用Web Server套件
from flask import Flask, request, abort, jsonify

# 載入json處理套件
import json

# 外部連結自動生成套件
from flask_ngrok import run_with_ngrok

# 從linebot 套件包裡引用 LineBotApi 與 WebhookHandler 類別
from linebot import (
    LineBotApi, WebhookHandler
)

# 引用無效簽章錯誤
from linebot.exceptions import (
    InvalidSignatureError
)

In [95]:
'''
建置主程序app

建置handler與 line_bot_api

準備好app
再準備 handler, 用來接收 Line 傳來的一切事務，順便做驗證，就像小秘書，過濾信件
準備 line_bot_api，有些業務要跟 Line 溝通，都會透過 line_bot_api，回信、取個資。

'''

# 設定Server啟用細節
app = Flask(__name__,static_url_path = "/material" , static_folder = "./material/")
run_with_ngrok(app)

# 生成實體物件
line_bot_api = LineBotApi("iRuogYf3cHsttNKwqUwl0phQV4fNuhmRPyVenE5DB5ArJHSLH/jYuJr31y7+DgGWRZJUSzr5i8NsvbKUcy7Xb9Akxys5GWdqm2bRcsbr/s03t4mpBO9UDnGcQG+BYHsxBFC6y36okKYWUg1qakCVSwdB04t89/1O/w1cDnyilFU=")
handler = WebhookHandler("de49f52f78bea9f42d8e2d4536da212c")

In [96]:
'''
建置主程序的API入口
  接受Line傳過來的消息
  並取出消息內容
  將消息內容存在google drive的檔案內
  並請handler 進行消息驗證與轉發

當流量消息 都給我們 ngrok.xxxx/(根目錄)，就會觸發 Line 消息的處理流程

'''

# 啟動server對外接口，使Line能丟消息進來
@app.route("/", methods=['POST'])
def callback():
    # get X-Line-Signature header value
    # line 的加密簽章
    signature = request.headers['X-Line-Signature']

    # get request body as text
    # 用戶傳來的內容
    body = request.get_data(as_text=True)
    print(body)

    # 記錄用戶log
    # 存在 google drive上，ai-event.log，ai-event.log
    f = open("/content/drive/MyDrive/ai-event.log", "a")
    f.write(body)
    f.close()

    # handle webhook body
    try:
        # 把消息交給 handler 做驗證
        # 會看是什麼樣的 event，做不同處理
        handler.handle(body, signature)
    except InvalidSignatureError:
        abort(400)

    return 'OK'

In [97]:
'''
菜單設定檔

    設定圖面大小、按鍵名與功能
    
'''

menuRawData = """
{
  "size": {
    "width": 2500,
    "height": 1686
  },
  "selected": true,
  "name": "區塊鏈自定義菜單",
  "chatBarText": "查看更多資訊",
  "areas": [
    {
      "bounds": {
        "x": 115,
        "y": 120,
        "width": 950,
        "height": 550
      },
      "action": {
        "type": "message",
        "text": "[::text:]吃飯"
      }
    },
    {
      "bounds": {
        "x": 115,
        "y": 940,
        "width": 950,
        "height": 550
      },
      "action": {
        "type": "message",
        "text": "[::text:]自由"
      }
    },
    {
      "bounds": {
        "x": 1345,
        "y": 120,
        "width": 950,
        "height": 550
      },
      "action": {
        "type": "message",
        "text": "[::text:]是否"
      }
    },
    {
      "bounds": {
        "x": 1345,
        "y": 940,
        "width": 950,
        "height": 550
      },
      "action": {
        "type": "message",
        "text": "[::text:]幹話"
      }
    }
    
  ]
}
"""

In [98]:
'''

載入前面的圖文選單設定，

並要求line_bot_api將圖文選單上傳至Line
    

'''

from linebot.models import RichMenu
import requests
import json
# 讀取圖文選單設定檔
menuJson=json.loads(menuRawData)
# 用 line_bot_api 的 create_rich_menu 方法 創造圖文選單 id
# rich_menu : RichMenu
# new_from_json_dict 可以從 json 裡面生成 RichMenu 物件
lineRichMenuId = line_bot_api.create_rich_menu(rich_menu=RichMenu.new_from_json_dict(menuJson))
print(lineRichMenuId)

richmenu-b018e34279eff3440bebc1e0877038fe


In [99]:
'''

將先前準備的菜單照片，以Post消息寄發給Line

    載入照片
    要求line_bot_api，將圖片傳到先前的圖文選單id


'''


uploadImageFile=open("abc.jpg",'rb')

setImageResponse = line_bot_api.set_rich_menu_image(lineRichMenuId,'image/jpeg',uploadImageFile)

print(setImageResponse)

None


In [100]:

'''

用戶關注 => FollowEvent

撰寫用戶關注時，我們要處理的商業邏輯

1. 取得用戶個資，並存回伺服器
2. 回應用戶，歡迎用的文字消息

'''

# 載入Follow事件
from linebot.models.events import (
    FollowEvent
)

from linebot.models import(
    TextSendMessage, ImageSendMessage
)

# 告知 handler，如果收到 FollowEvent，則做下面的方法處理
# 告知 handler，收到 FollowEvent 的話，就執行下面的方法
@handler.add(FollowEvent)
def reply_text_and_get_user_profile(event):
    
    # 取出消息內User的資料
    # 用 line_bot_api 跟 line 溝通，取得個資 get_profile(event.source.user_id)
    # event 是每次傳來的消息
    # source 來源
    # user_id 發消息的用戶 id
    # event.source.user_id
    user_profile = line_bot_api.get_profile(event.source.user_id)
        
     # 把 user_profile 用 var() 處理成字典類型(dict)
     # json.dumps(): json.dumps()用於將dict型別的資料轉成str，
     # 因為如果直接將dict型別的資料寫入json檔案中會發生報錯，因此在將資料寫入時需要用到該函式。
     # 將用戶資訊存在檔案內
     # 存在 google drive 的 user.txt
    with open("/content/drive/MyDrive/users.txt", "a") as myfile:
        myfile.write(json.dumps(vars(user_profile),sort_keys=True))
        myfile.write('\n')
    
    # 回覆文字消息與圖片消息
    # 請 line_bot_api 回應消息給 line，line 轉發回去給用戶
    # reply_message(reply_token)
    # 每一次用戶傳給 line，line 轉傳過來的時候，附上這個票券，我們必須拿這個票券，跟 line 回應
    line_bot_api.reply_message(
      event.reply_token,
      [TextSendMessage('''你好，歡迎使用選擇障礙神器
我將解決你的選擇問題!''')]
    )

    linkResult=line_bot_api.link_rich_menu_to_user(event.source.user_id, lineRichMenuId)

    print(linkResult)

    print(event.source.user_id)
    
# def Link(event):
#   '''

#   將選單綁定到特定用戶身上
#       取出上面得到的菜單Id及用戶id
#       要求line_bot_api告知Line，將用戶與圖文選單做綁定

#   '''

#   # https://api.line.me/v2/bot/user/{userId}/richmenu/{richMenuId}


#   linkResult=line_bot_api.link_rich_menu_to_user(event.source.user_id, lineRichMenuId)

#   print(linkResult)

#   print(event.source.user_id)

In [101]:
# 運行主程序
app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://e9c24d4d2cfa.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [25/May/2021 12:58:15] "POST / HTTP/1.1" 200 -


{"destination":"Uce979b9786733f30f88ad4eb1d9e8125","events":[{"type":"unfollow","timestamp":1621947494559,"source":{"type":"user","userId":"U69b5bef808bc39b5bb76b7d5e17b7a82"},"mode":"active"}]}
{"destination":"Uce979b9786733f30f88ad4eb1d9e8125","events":[{"type":"follow","timestamp":1621947495429,"source":{"type":"user","userId":"U69b5bef808bc39b5bb76b7d5e17b7a82"},"replyToken":"4983ef058832422f91997415a51df04b","mode":"active"}]}


...

{"destination":"Uce979b9786733f30f88ad4eb1d9e8125","events":[{"type":"message","message":{"type":"text","id":"14112652492238","text":"[::text:]幹話"},"timestamp":1621947534533,"source":{"type":"user","userId":"U69b5bef808bc39b5bb76b7d5e17b7a82"},"replyToken":"d9da1525a1904ee6ba1f76e4c1955fed","mode":"active"}]}


127.0.0.1 - - [25/May/2021 12:58:55] "POST / HTTP/1.1" 200 -


{"destination":"Uce979b9786733f30f88ad4eb1d9e8125","events":[{"type":"message","message":{"type":"text","id":"14112652537712","text":"[::text:]自由"},"timestamp":1621947535026,"source":{"type":"user","userId":"U69b5bef808bc39b5bb76b7d5e17b7a82"},"replyToken":"388e042f22604281a2f1cb7b3b22b82f","mode":"active"}]}


127.0.0.1 - - [25/May/2021 12:58:55] "POST / HTTP/1.1" 200 -


{"destination":"Uce979b9786733f30f88ad4eb1d9e8125","events":[{"type":"message","message":{"type":"text","id":"14112652566111","text":"[::text:]吃飯"},"timestamp":1621947535326,"source":{"type":"user","userId":"U69b5bef808bc39b5bb76b7d5e17b7a82"},"replyToken":"6ab3c1063a5347cfbab02c0a5912cff5","mode":"active"}]}
